In [1]:
import numpy as np
import pandas as pd

In [144]:
akas_df = pd.read_csv('./data/title.akas.tsv', na_values='\\N', low_memory=False)
ratings_df = pd.read_csv('./data/title.ratings.tsv', sep='\t', na_values='\\N', low_memory=False)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 68, saw 3


In [83]:
basics_df = pd.read_csv('./data/title.basics.tsv', sep='\t', na_values='\\N', low_memory=False)

In [97]:
def filter_years(data, start_year, end_year):
    common_years = set(data['startYear'].dropna().unique().astype(int))

    if start_year and end_year:
        common_years = common_years.intersection(range(start_year, end_year + 1))

    if not common_years:
        raise ValueError("No common years found between the datasets.")

    processed_data = data[data['startYear'].isin(common_years)]
    
    return processed_data

In [98]:
def merge_data(basics_df: pd.DataFrame, ratings_df: pd.DataFrame, akas_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge the data from the dataframes.

    :param basics_df: pd.DataFrame: Data with basic information about the movies
    :param ratings_df: pd.DataFrame: Data with ratings of the movies
    :param akas_df: pd.DataFrame: Data with information about the different regions where the movies were presented

    :return: pd.DataFrame: Merged data from the dataframes
    """
    merged_df = akas_df.merge(basics_df, left_on='titleId', right_on='tconst')
    merged_df = merged_df.merge(ratings_df, on='tconst')

    return merged_df

In [112]:
def get_top_n_movies_per_country(movies_df, n):
    # Grupowanie filmów według regionu i liczenie liczby filmów w każdym regionie
    country_movie_counts = movies_df['region'].value_counts()

    # Wybieranie tylko tych krajów, które mają co najmniej n filmów
    valid_countries = country_movie_counts[country_movie_counts >= n].index
    valid_movies_df = movies_df[movies_df['region'].isin(valid_countries)]

    # Sortowanie filmów według ocen i liczby głosów
    valid_movies_df = valid_movies_df.sort_values(by=['region', 'averageRating', 'numVotes'], ascending=[True, False, False])

    # Grupy według regionów i wybór top n filmów
    top_n_movies_df = valid_movies_df.groupby('region').head(n)

    # Obliczanie średniej oceny dla top n filmów w każdym kraju
    top_n_ratings_df = top_n_movies_df.groupby('region').agg({
        'averageRating': 'mean',
        'numVotes': 'sum',
        'tconst': 'count'
    }).reset_index()

    top_n_ratings_df.columns = ['region', 'avg_rating', 'total_votes', 'film_count']
    return top_n_ratings_df

In [113]:
basics_df_processed = filter_years(basics_df, 2010, 2019)

In [101]:
akas_df = akas_df[['titleId', 'region']]
basics_df_processed = basics_df_processed[['tconst', 'titleType', 'primaryTitle', 'startYear']]
ratings_df = ratings_df[['tconst', 'averageRating', 'numVotes']]

In [102]:
merged_df = merge_data(basics_df_processed, ratings_df, akas_df)

movies_df = merged_df[merged_df['titleType'] == 'movie']

In [105]:
movies_df.head(50)

,titleId,region,tconst,titleType,primaryTitle,startYear,averageRating,numVotes
16,tt0069049,NaN,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
17,tt0069049,ES,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
18,tt0069049,BR,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
19,tt0069049,PT,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
20,tt0069049,HU,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
21,tt0069049,FR,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
22,tt0069049,RS,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
23,tt0069049,PL,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
24,tt0069049,IT,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066
25,tt0069049,VE,tt0069049,movie,The Other Side of the Wind,2018.0,6.7,8066


In [111]:
for n in [10, 20, 50, 100, 200]:
        top_n_ratings_df = get_top_n_movies_per_country(movies_df, n)
        top_n_ratings_df = top_n_ratings_df.sort_values(by='avg_rating', ascending=False)
        print(f"Top {n} countries based on top {n} movies:")
        print(top_n_ratings_df.head(10))

IndexError: Too many levels: Index has only 1 level, not 2

In [96]:
top_20_countries = get_top_n(res, 20)
top_20_countries

['DM',
 'IM',
 'SL',
 'OM',
 'ZW',
 'UG',
 'RW',
 'AN',
 'MH',
 'TZ',
 'NC',
 'ET',
 'QA',
 'TN',
 'MV',
 'MG',
 'ER',
 'DDDE',
 'XPI',
 'JO']

In [ ]:
# akas_df = akas_df[['titleId', 'region']]
# basics_df = basics_df[['tconst', 'titleType', 'primaryTitle', 'startYear']]
# 
# merged_df = akas_df.merge(basics_df, left_on='titleId', right_on='tconst')
# merged_df = merged_df.merge(ratings_df, on='tconst')
# 
# movies_df = merged_df[merged_df['titleType'] == 'movie']



In [55]:
# def process_data_analysis1(basis_data, akas_data, ratings_data, start, end):
#     films = basis_data[basis_data['titleType'] == 'movie']
#     films_selected_cols = films.filter(items=['primaryTitle', 'startYear', 'runtimeMinutes', 'genres'])
#     films_selected_cols.dropna(subset=['startYear'], inplace=True)
#     films_selected_cols['startYear'] = films_selected_cols['startYear'].astype(int)
#     films_processed = films_selected_cols[(films_selected_cols['startYear'] >= start) & (films_selected_cols['startYear'] <= end)]
#     
#     print(len(films_processed))
#     
#     akas_data = akas_data[akas_data['isOriginalTitle'] == 1]
#     akas_selected_cols = akas_data.filter(items=['region'])
#     
#     print(len(akas_selected_cols))
#     
#     print(len(ratings_data))
#     merged_data = pd.merge(films_processed, akas_selected_cols, left_index=True, right_index=True)
# 
#     merged_data = pd.merge(merged_data, ratings_data, left_index=True, right_index=True)
#     
#     print(len(merged_data))
#     
#     merged_data.dropna(subset=['region', 'averageRating'], inplace=True)
#     
#     print(len(merged_data))
    
    # return merged_data

In [114]:
import data_analysis.data_processing as dp
from data_analysis.analysis import perform_task_1

In [115]:
basics = dp.load_data('./data/title.basics.tsv')
akas = dp.load_data('./data/title.akas.tsv')
ratings = dp.load_data('./data/title.ratings.tsv')

In [121]:
akas_selected, basis_selected, ratings_selected = dp.keep_interesting_columns(basics, ratings, akas)

In [122]:
basis_selected.head()

,tconst,titleType,primaryTitle,startYear
0,tt0000001,short,Carmencita,1894.0
1,tt0000002,short,Le clown et ses chiens,1892.0
2,tt0000003,short,Pauvre Pierrot,1892.0
3,tt0000004,short,Un bon bock,1892.0
4,tt0000005,short,Blacksmith Scene,1893.0


In [123]:
basis_filtered = dp.filter_years(basis_selected, 2000, 2020)

In [124]:
merged_data = dp.merge_data(basis_filtered, ratings_selected, akas_selected)
merged_data = merged_data[merged_data['titleType'] == 'movie']

In [161]:
merged_data[merged_data['region'] == 'FR'].sort_values(by='averageRating', ascending=False).head(100)

,titleId,region,tconst,titleType,primaryTitle,startYear,averageRating,numVotes,alpha-2,name
218740,tt22046656,FR,tt22046656,movie,Wild Gift,2022.0,10.0,8,FR,France
214062,tt18372080,FR,tt18372080,movie,The World Cruzer,2022.0,10.0,6,FR,France
205260,tt14485794,FR,tt14485794,movie,Éoliennes: du rêve aux réalités,2021.0,9.4,95,FR,France
219724,tt22875818,FR,tt22875818,movie,Highwaymen Bloodlust,2022.0,9.4,8,FR,France
213556,tt18244676,FR,tt18244676,movie,Pyramide Le Grand Virage,2022.0,9.3,6,FR,France
...,...,...,...,...,...,...,...,...,...,...
208747,tt15296568,FR,tt15296568,movie,I Am More,2021.0,8.3,16,FR,France
219641,tt22813298,FR,tt22813298,movie,Sons of the Wind,2022.0,8.2,6,FR,France
199525,tt13280182,FR,tt13280182,movie,The Art of Sin,2020.0,8.2,10,FR,France
223054,tt8637504,FR,tt8637504,movie,Disclosure,2020.0,8.2,6097,FR,France


In [141]:
merged_data = merged_data.drop_duplicates(subset=['region', 'tconst', 'startYear', 'averageRating', 'numVotes'])

country_movie_counts = merged_data['region'].value_counts()

In [142]:
country_movie_counts

region
US     89658
GB     68027
RU     32929
XWW    31896
DE     31816
       ...  
LS         1
XAU        1
LC         1
GI         1
WS         1
Name: count, Length: 221, dtype: int64

In [145]:


from yaml._yaml import ParserError


def load_data(file_path: str) -> pd.DataFrame:
    """
    Load the data from the CSV file.

    :param file_path: str: Path to the CSV file with the data

    :return pd.DataFrame: Data from the CSV file
    """
    try:
        data = pd.read_csv(file_path, low_memory=False, na_values='\\N')
    except ParserError:
        data = pd.read_csv(file_path, sep='\t', low_memory=False, na_values='\\N')

    return data

In [146]:
basics = dp.load_data('./data/title.basics.tsv')

In [170]:
countries = load_data('./data/countries.csv')

In [171]:
def keep_interesting_columns(
        basics_df: pd.DataFrame,
        ratings_df: pd.DataFrame,
        akas_df: pd.DataFrame,
        countries_df: pd.DataFrame,
):
    """
    Filter the dataframes to keep only the interesting columns.

    :param basics_df: pd.DataFrame:
        Data with basic information about the movies
    :param ratings_df: pd.DataFrame:
        Data with ratings of the movies
    :param akas_df: pd.DataFrame:
        Data with information about the different regions where the movies were presented
    :param countries_df: pd.DataFrame:
        Data with information about the names of the countries based on the region codes

    :return: Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]: Filtered dataframes
    """
    basics_df = basics_df[['tconst', 'titleType', 'primaryTitle', 'startYear']]
    ratings_df = ratings_df[['tconst', 'averageRating', 'numVotes']]
    akas_df = akas_df[['titleId', 'region']]
    countries_df = countries_df[['alpha-2', 'name']]

    return basics_df, ratings_df, akas_df, countries_df

In [172]:
basics_selected, ratings_selected, akas_selected, countries_selected = keep_interesting_columns(
        basics, ratings, akas, countries)

In [154]:
countries_selected

,alpha-2,name
0,AF,Afghanistan
1,AX,Åland Islands
2,AL,Albania
3,DZ,Algeria
4,AS,American Samoa
...,...,...
244,WF,Wallis and Futuna
245,EH,Western Sahara
246,YE,Yemen
247,ZM,Zambia


In [155]:
def merge_data(
        basics_df: pd.DataFrame,
        ratings_df: pd.DataFrame,
        akas_df: pd.DataFrame,
        countries_df: pd.DataFrame,
) -> pd.DataFrame:
    """
    Merge the data from the dataframes.

    :param basics_df: pd.DataFrame:
        Data with basic information about the movies
    :param ratings_df: pd.DataFrame:
        Data with ratings of the movies
    :param akas_df: pd.DataFrame:
        Data with information about the different regions where the movies were presented
    :param countries_df: pd.DataFrame:
        Data with information about the names of the countries based on the region codes

    :return: pd.DataFrame: Merged data from the dataframes
    """
    merged_df = akas_df.merge(basics_df, left_on='titleId', right_on='tconst')
    merged_df = merged_df.merge(ratings_df, on='tconst')
    merged_df = merged_df.merge(countries_df, left_on='region', right_on='alpha-2')

    return merged_df

In [173]:
# basics_filtered = dp.filter_years(basics_selected, 2020, 2022)
    
merged_data = merge_data(basics_filtered, ratings_selected, akas_selected, countries_selected)
merged_data = merged_data[merged_data['titleType'] == 'movie']

In [160]:
merged_data.head()

,titleId,region,tconst,titleType,primaryTitle,startYear,averageRating,numVotes,alpha-2,name
0,tt0013274,NaN,tt0013274,movie,Istoriya grazhdanskoy voyny,2021.0,6.8,69,NaN,Namibia
1,tt0013274,NaN,tt0013274,movie,Istoriya grazhdanskoy voyny,2021.0,6.8,69,NaN,Namibia
3,tt0062336,NaN,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,2020.0,6.4,188,NaN,Namibia
4,tt0062336,NaN,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,2020.0,6.4,188,NaN,Namibia
5,tt0096235,NaN,tt0096235,movie,Taxi Killer,2022.0,5.6,73,NaN,Namibia


In [176]:
def perform_task_1(merged_df: pd.DataFrame, numbers=(10, 20, 50, 100, 200)) -> None:
    """
    Perform the task 1 analysis.

    :param merged_df: pd.DataFrame: Merged dataframe with the movie data
    :param numbers: tuple: Tuple with the numbers of top movies to consider
    :return: None
    """
    for n in numbers:
        top_n_ratings_df = get_top_n_movies_per_country(merged_df, n)
        top_n_ratings_df = top_n_ratings_df.sort_values(by='avg_rating', ascending=False)
        print('-' * 50)
        print(f"Top 10 countries based on top {n} films: ")
        print(top_n_ratings_df.head(10))


def get_top_n_movies_per_country(movies_df, n) -> pd.DataFrame:
    """
    Get the average rating of the top n movies per country.
    In case of same rating, the number of votes is used as a tiebreaker.
    The process takes into account only countries with at least n movies
    (in order to have comparable and fair rankings)

    :param movies_df: pd.DataFrame: Dataframe with the movies data
    :param n: int: Number of top movies to choose per country

    :return: pd.DataFrame: Dataframe with the average rating of the top n movies per country
    """
    movies_df = movies_df.drop_duplicates(
        subset=['region', 'tconst', 'startYear', 'averageRating', 'numVotes'],
        keep='first',
    )
    country_movie_counts = movies_df['region'].value_counts()
    countries_with_at_least_n_films = country_movie_counts[country_movie_counts >= n].index
    valid_df = movies_df[movies_df['region'].isin(countries_with_at_least_n_films)]
    valid_df = valid_df.sort_values(
        by=['name', 'region', 'averageRating', 'numVotes'],
        ascending=[True, True, False, False],
    )

    top_n_movies_df = valid_df.groupby('region').head(n)
    top_n_ratings_df = top_n_movies_df.groupby(['name', 'region']).agg({
        'averageRating': 'mean',
        'numVotes': 'sum',
        'tconst': 'count'
    }).reset_index()

    top_n_ratings_df.columns = ['country_name', 'country_code', 'avg_rating', 'total_votes', 'film_count']
    return top_n_ratings_df


In [177]:
perform_task_1(merged_data)

--------------------------------------------------
Top 10 countries based on top 10 films: 
                        country_name country_code  avg_rating  total_votes  \
103         United States of America           US       10.00           96   
102  United Kingdom of Great Britain           GB        9.89          180   
16                            Canada           CA        9.87          262   
42                             India           IN        9.86          617   
48                             Italy           IT        9.77          278   
4                          Australia           AU        9.74          220   
12                            Brazil           BR        9.67          238   
96                          Thailand           TH        9.64          554   
34                           Germany           DE        9.62          314   
89                             Spain           ES        9.61          572   

     film_count  
103          10  
102          

In [2]:
from data_analysis.analysis import perform_task_1
from data_analysis.data_processing import load_data, keep_interesting_columns, merge_data, fi

basics = load_data('./data/title.basics.tsv')
ratings = load_data('./data/title.ratings.tsv')
akas = load_data('./data/title.akas.tsv')
countries = load_data('./data/countries.csv')


KeyError: "None of [Index(['tconst', 'averageRating', 'numVotes'], dtype='object')] are in the [columns]"

In [5]:
ratings.columns

Index(['tconst\taverageRating\tnumVotes'], dtype='object')

In [14]:
import pandas as pd
ratings = pd.read_csv('./data/title.ratings.tsv', sep='\t', na_values='\\N', low_memory=False)

In [15]:
basics_selected, ratings_selected, akas_selected, countries_selected = keep_interesting_columns(
        basics, ratings, akas, countries,
)

In [16]:
from data_analysis.data_processing import filter_years, merge_data
basics_filtered = filter_years(basics_selected, 1990, 2000)
merged_data = merge_data(basics_filtered, ratings_selected, akas_selected, countries_selected)
merged_data = merged_data[merged_data['titleType'] == 'movie']


In [25]:
merged_data = merged_data.dropna(subset=['region'])

In [26]:
movies_df = merged_data.drop_duplicates(
        subset=['region', 'tconst', 'startYear', 'averageRating', 'numVotes'],
        keep='first',
    )
movies_df

,titleId,region,tconst,titleType,primaryTitle,startYear,averageRating,numVotes,alpha-2,name
156361,tt0038086,DE,tt0038086,movie,Shiva und die Galgenblume,1993.0,7.0,27,DE,Germany
156362,tt0059097,DE,tt0059097,movie,Just Don't Think I'll Cry,1990.0,6.6,63,DE,Germany
156363,tt0059325,DE,tt0059325,movie,Born in '45,1990.0,6.5,270,DE,Germany
156364,tt0059900,DE,tt0059900,movie,"Wenn du groß bist, lieber Adam",1990.0,6.4,42,DE,Germany
156365,tt0070080,DE,tt0070080,movie,Barnen från Jordbro,1996.0,7.2,50,DE,Germany
...,...,...,...,...,...,...,...,...,...,...
582695,tt0390072,BH,tt0390072,movie,The Barrier,1990.0,5.2,19,BH,Bahrain
582705,tt2032431,MV,tt2032431,movie,Dheriyaa,1994.0,7.2,13,MV,Maldives
582708,tt3429952,SD,tt3429952,movie,The Sheikh's Blessing,1998.0,7.4,13,SD,Sudan
582710,tt3990150,MM,tt3990150,movie,Thu Kyun Ma Khan Py,1997.0,6.4,10,MM,Myanmar


In [28]:
merged_data[merged_data['tconst'] == 'tt0015414']

,titleId,region,tconst,titleType,primaryTitle,startYear,averageRating,numVotes,alpha-2,name
183160,tt0015414,FR,tt0015414,movie,La tierra de los toros,2000.0,5.4,17,FR,France
183161,tt0015414,FR,tt0015414,movie,La tierra de los toros,2000.0,5.4,17,FR,France
205460,tt0015414,ES,tt0015414,movie,La tierra de los toros,2000.0,5.4,17,ES,Spain


In [29]:
perform_task_1(merged_data) 

--------------------------------------------------
Top 10 countries based on top 10 films: 
                       country_name  avg_rating  total_votes  film_count
87         United States of America        9.63          416          10
86  United Kingdom of Great Britain        9.46      2914914          10
36                            India        9.29      2914785          10
64                      Philippines        9.25      4370342          10
43                            Japan        9.21      6603679          10
34                          Hungary        9.18      6584054          10
16                           Canada        9.17      4375552          10
29                           France        9.14      4370280          10
31                          Germany        9.11      6599461          10
52                           Mexico        9.05     13243617          10
--------------------------------------------------
Top 10 countries based on top 20 films: 
             